# `AnswerBuilder`

In [1]:
%load_ext autoreload
%autoreload 2

## On its own

In [2]:
from haystack.components.builders import AnswerBuilder

In [3]:
builder = AnswerBuilder(pattern="Answer: (.*)")
res = builder.run(query="What's the answer?", replies=["This is an argument. Answer: This is the answer."])

In [4]:
res

{'answers': [GeneratedAnswer(data='This is the answer.', query="What's the answer?", documents=[], meta={})]}

## In pipeline

In [5]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack_integrations.components.generators.ollama import OllamaGenerator
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.utils import Secret

In [6]:
prompt_template = """
    Given these documents, answer the question.\nDocuments:
    {% for doc in documents %}
        {{ doc.content }}
    {% endfor %}

    \nQuestion: {{query}}
    \nAnswer:
    """

In [7]:
p = Pipeline()
p.add_component(instance=InMemoryBM25Retriever(document_store=InMemoryDocumentStore()), name="retriever")
p.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
p.add_component(instance=OllamaGenerator(
    model="zephyr",
    streaming_callback=lambda chunk: print(chunk.content, end="", flush=True),
    url = "http://localhost:11434",
    generation_kwargs={
        "num_predict": 100,
        "temperature": 0.9}), name="llm")
p.add_component(instance=AnswerBuilder(), name="answer_builder")

In [8]:
p.connect("retriever", "prompt_builder.documents")
p.connect("prompt_builder", "llm")
p.connect("llm.replies", "answer_builder.replies")
p.connect("llm.meta", "answer_builder.meta")
p.connect("retriever", "answer_builder.documents")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: PromptBuilder
  - llm: OllamaGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> answer_builder.replies (List[str])
  - llm.meta -> answer_builder.meta (List[Dict[str, Any]])

In [9]:
query = "Thủ đô của Việt Nam là gì?"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)

Hà Nội là thủ đô của Việt Nam. (Được biết từ văn bản trang thứ hai)


Explanation:
The second document provides the answer to the question. It states that Hà Nội is the capital of Vietnam. Therefore, the answer to the question is Hà Nội.

In [10]:
query = "Kubernetes là gì?"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)


Kubernetes (k8s) là một phần mềm open-source dựa trên Go-language, do Google phát triển và phân bổ tới cộng đồng mã nguồn mở. Nó cho phép quản lý và tự động hóa quyết chiến chấp thuận sử dụ

In [11]:
query = "Cho tôi biết các ngôn ngữ mà Zephyr model có thể hỗ trợ"
result = p.run(
    {
        "retriever": {"query": query},
        "prompt_builder": {"query": query},
        "answer_builder": {"query": query},
    }
)


Tôi biết các ngôn ngữ mà dòng máy bay Zephyr có thể hỗ trợ là:
- Tiếng Anh (English)
- Tiếng Pháp (French)
- Tiếng Đức (German)
- Tiếng Tập Thận (Italian)
- Tiếng Nhật (Japanese)
